In [1]:
import scipy.sparse
from scipy.sparse import coo_matrix, csc_matrix, csr_matrix
import numpy as np

# Slicing

In [15]:
A = csr_matrix([[1, 0, 1], [1, 1, 1], [0, 1, 0]])
s = np.array([2, 0, 1, 2, 0, 1])
print("mat\n", A.A)
print("indptr", A.indptr)
print("indices", A.indices)

start_index = np.take(A.indptr, s)
print("start", start_index)
end_index = np.take(A.indptr, s+1)
print("end", end_index)
length = end_index - start_index
print("length", length)
new_indptr = np.concatenate([[0], np.cumsum(length)])
print("new indptr", new_indptr)
#slices = tuple(map(slice, start_index, end_index)) # iterative
new_indices = np.concatenate([A.indices[np.s_[i:j]] for (i, j) in zip(start_index, end_index)])
print("new indices", new_indices)

new_A = csr_matrix((np.ones_like(new_indices), new_indices, new_indptr))
print("new mat \n", new_A.A)


mat
 [[1 0 1]
 [1 1 1]
 [0 1 0]]
indptr [0 2 5 6]
indices [0 2 0 1 2 1]
start [5 0 2 5 0 2]
end [6 2 5 6 2 5]
length [1 2 3 1 2 3]
new indptr [ 0  1  3  6  7  9 12]
new indices [1 0 2 0 1 2 1 0 2 0 1 2]
new mat 
 [[0 1 0]
 [1 0 1]
 [1 1 1]
 [0 1 0]
 [1 0 1]
 [1 1 1]]


In [2]:
import tracemalloc
tracemalloc.start()
snapshot = []
def numpy_slice_sparse_matrix(A, s, default_value=None):
    """
    A: csr sparse matrix
    s: row indices
    default_value: default the new sparse matrix with this single value
    """
    #snapshot.append(tracemalloc.take_snapshot()) # at start
    A = A.tocsr()
    start_index = np.take(A.indptr, s)
    end_index = np.take(A.indptr, s+1)
    new_indptr = np.concatenate([[0], np.cumsum(end_index - start_index)])
    #snapshot.append(tracemalloc.take_snapshot()) # after finding out start end indices
    # This could still be efficient if the slice length is not very long
    # Hence efficient for sparse matrices
    new_indices = np.concatenate([A.indices[i:j] for (i, j) in zip(start_index, end_index)])
    #snapshot.append(tracemalloc.take_snapshot()) # after finding new indices
    if default_value is None:
        # Use original value
        new_values = np.concatenate([A.data[i:j] for (i, j) in zip(start_index, end_index)])
    else:
        new_values = default_value * np.ones_like(new_indices)
    #snapshot.append(tracemalloc.take_snapshot()) # after finding new values

    # free some memory
    del start_index
    del end_index 
    new_A = csr_matrix((new_values, new_indices, new_indptr))
    #snapshot.append(tracemalloc.take_snapshot()) # after making the new sparse matrix
    return new_A

#A = csr_matrix([[1, 0, 1], [1, 1, 1], [0, 1, 0]])
#s = np.array([2, 0, 1, 2, 0, 1])
#new_A = numpy_slice_sparse_matrix(A, s)
#new_A.A

A = coo_matrix((np.ones(int(2E6)), 
                (np.random.randint(0, 150000, size=int(2E6)), 
                 np.random.randint(0, 150000, size=int(2E6))))).tocsr()
s = np.arange(150000)
np.random.shuffle(s)

new_A = numpy_slice_sparse_matrix(A, s, default_value=None)

In [3]:
from memory_profiler import memory_usage
def shift_A():
    A = coo_matrix((np.ones(int(2E6)), 
                    (np.random.randint(0, 150000, size=int(2E6)), 
                     np.random.randint(0, 150000, size=int(2E6))))).tocsr()
    s = np.arange(150000)
    np.random.shuffle(s)
    new_A = numpy_slice_sparse_matrix(A, s)
mem = memory_usage(proc=shift_A)
print(mem)

[164.015625, 164.12109375, 189.53125, 194.66015625, 194.66015625, 196.95703125, 200.19921875, 203.83203125, 207.88671875, 210.39453125, 214.97265625, 219.69140625, 191.3046875, 167.7109375, 168.1328125, 168.91796875, 171.66796875, 176.47265625, 181.58203125, 193.640625, 210.234375, 192.08203125, 176.58203125]


In [54]:
top_stats = snapshot[3].compare_to(snapshot[1], 'lineno')
print("[ Top 10 differences ]")
for stat in top_stats[:10]:
    print(stat)

[ Top 10 differences ]
<__array_function__ internals>:5: size=72.1 MiB (+22.9 MiB), count=32 (+6), average=2307 KiB
<ipython-input-49-8f494043b31d>:22: size=712 B (+712 B), count=9 (+9), average=79 B
/Users/edward/opt/anaconda3/lib/python3.8/site-packages/ipykernel/kernelbase.py:291: size=7208 B (-496 B), count=14 (-1), average=515 B
<ipython-input-49-8f494043b31d>:25: size=472 B (+472 B), count=1 (+1), average=472 B
<ipython-input-49-8f494043b31d>:15: size=0 B (-472 B), count=0 (-1)
/Users/edward/opt/anaconda3/lib/python3.8/signal.py:48: size=7608 B (-464 B), count=15 (-1), average=507 B
<ipython-input-49-8f494043b31d>:18: size=464 B (+464 B), count=1 (+1), average=464 B
/Users/edward/opt/anaconda3/lib/python3.8/tracemalloc.py:397: size=1760 B (+304 B), count=25 (+6), average=70 B
/Users/edward/opt/anaconda3/lib/python3.8/codeop.py:136: size=6716 B (-248 B), count=85 (-4), average=79 B
/Users/edward/opt/anaconda3/lib/python3.8/tracemalloc.py:532: size=60.2 KiB (+224 B), count=1151 (+4

In [6]:
import tracemalloc
tracemalloc.start()
new_A = A[s, :]
snapshot = tracemalloc.take_snapshot()
top_stats = snapshot.statistics('lineno')
print("[ Top 10 differences ]")
for stat in top_stats[:10]:
    print(stat)

[ Top 10 differences ]
<__array_function__ internals>:5: size=22.9 MiB, count=13, average=1803 KiB
/Users/edward/opt/anaconda3/lib/python3.8/site-packages/scipy/sparse/compressed.py:694: size=15.3 MiB, count=3, average=5208 KiB
/Users/edward/opt/anaconda3/lib/python3.8/site-packages/scipy/sparse/compressed.py:693: size=7812 KiB, count=2, average=3906 KiB
<ipython-input-2-80e8b9ec04e9>:40: size=1172 KiB, count=2, average=586 KiB
/Users/edward/opt/anaconda3/lib/python3.8/site-packages/scipy/sparse/compressed.py:689: size=586 KiB, count=3, average=195 KiB
/Users/edward/opt/anaconda3/lib/python3.8/site-packages/scipy/sparse/coo.py:402: size=586 KiB, count=2, average=293 KiB
/Users/edward/opt/anaconda3/lib/python3.8/linecache.py:137: size=330 KiB, count=3174, average=106 B
<frozen importlib._bootstrap_external>:580: size=311 KiB, count=3283, average=97 B
/Users/edward/opt/anaconda3/lib/python3.8/posixpath.py:368: size=112 KiB, count=908, average=126 B
/Users/edward/opt/anaconda3/lib/python3

In [4]:
from memory_profiler import memory_usage
def shift_A():
    A = coo_matrix((np.ones(int(2E6)), 
                    (np.random.randint(0, 150000, size=int(2E6)), 
                     np.random.randint(0, 150000, size=int(2E6))))).tocsr()
    s = np.arange(150000)
    np.random.shuffle(s)
    new_A = A[s, :]
mem = memory_usage(proc=shift_A)
print(mem)

[174.60546875, 174.60546875, 179.2890625, 189.8671875, 189.8671875, 189.97265625]


In [1]:
indices = np.array([0, 2, 0, 1, 2, 1]*30000)
indices

array([0, 2, 0, ..., 1, 2, 1])

In [7]:
start_index = np.random.randint(0, len(indices), size=int(1E6))
end_index = np.minimum(start_index + np.random.randint(1, 500, size=int(1E6)), len(indices))

In [11]:
np.concatenate([indices[i:j] for i, j in zip(start_index, end_index)]).shape

(249776513,)

Conclusion: The `A[s, :]` operation is okay. It's both memory efficent and fast

In [9]:
A_i = coo_matrix((np.ones(int(2E6)), 
                (np.random.randint(0, 150000, size=int(2E6)), 
                 np.random.randint(0, 150000, size=int(2E6))))).tocsr()
s = np.arange(150000)
np.random.shuffle(s)
A_i_1 = A_i[s, :]

In [10]:
A_i.multiply(A_i_1)

<150000x150000 sparse matrix of type '<class 'numpy.float64'>'
	with 218 stored elements in Compressed Sparse Row format>